In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [3]:
data_dir = '/content/drive/MyDrive/PLSU/'

In [4]:
img_size = (224, 224)
batch_size = 8

In [5]:
data_df = pd.read_excel(data_dir + 'label.xlsx', engine='openpyxl')
data_df

,File Number,life,head,heart,label
0,1,0,0,0,000
1,2,0,0,0,000
2,5,0,1,0,010
3,12,0,0,1,001
4,13,0,1,0,010
...,...,...,...,...,...
1031,5354,1,0,1,101
1032,5366,0,0,0,000
1033,5367,0,0,0,000
1034,5372,0,-,1,0-1


In [6]:
data_df = data_df[(data_df['life'] != '-') & (data_df['life'] != '')]
data_df = data_df[(data_df['head'] != '-') & (data_df['head'] != '')]
data_df = data_df[(data_df['heart'] != '-') & (data_df['heart'] != '')]
data_df

,File Number,life,head,heart,label
0,1,0,0,0,000
1,2,0,0,0,000
2,5,0,1,0,010
3,12,0,0,1,001
4,13,0,1,0,010
...,...,...,...,...,...
1030,5353,0,0,1,001
1031,5354,1,0,1,101
1032,5366,0,0,0,000
1033,5367,0,0,0,000


In [7]:
skipped_img = [68, 206, 415, 597, 601, 673, 792, 951, 1013, 1054, 1068, 1114, 1324, 1362, 1495, 1872, 1908, 1952, 2367, 2485, 2557, 2568, 2603, 2723, 2782, 2798, 2818, 2868, 3064, 3079, 3125, 3171, 3202, 3500, 3509, 3515, 3558, 4190, 4460, 5097, 5117, 5138, 5208, 310, 459, 817, 2145, 4676]
data_df = data_df[~data_df['File Number'].isin(skipped_img)]
data_df

,File Number,life,head,heart,label
0,1,0,0,0,000
1,2,0,0,0,000
2,5,0,1,0,010
3,12,0,0,1,001
4,13,0,1,0,010
...,...,...,...,...,...
1030,5353,0,0,1,001
1031,5354,1,0,1,101
1032,5366,0,0,0,000
1033,5367,0,0,0,000


In [8]:
def reconstruct_file_name(file_number):
    return 'image' + str(file_number) + '.png'

data_df['file'] = data_df['File Number'].apply(reconstruct_file_name)
del data_df['File Number']

data_df

<ipython-input-8-c51d8c8247da>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df['file'] = data_df['File Number'].apply(reconstruct_file_name)


,life,head,heart,label,file
0,0,0,0,000,image1.png
1,0,0,0,000,image2.png
2,0,1,0,010,image5.png
3,0,0,1,001,image12.png
4,0,1,0,010,image13.png
...,...,...,...,...,...
1030,0,0,1,001,image5353.png
1031,1,0,1,101,image5354.png
1032,0,0,0,000,image5366.png
1033,0,0,0,000,image5367.png


In [13]:
data_df = data_df.astype('string')
print(data_df['life'].dtype)

string


In [14]:
print(data_df['life'].value_counts())
print(data_df['head'].value_counts())
print(data_df['heart'].value_counts())

0    600
1    348
Name: life, dtype: Int64
0    665
1    283
Name: head, dtype: Int64
0    507
1    441
Name: heart, dtype: Int64


In [15]:
train_df = data_df.sample(frac=0.8, random_state=42)
valid_df = data_df.drop(train_df.index).sample(frac=0.5, random_state=42)
test_df = data_df.drop(train_df.index).drop(valid_df.index)
print(len(train_df), len(valid_df), len(test_df))

758 95 95


In [16]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
)

In [17]:
data_generator = ImageDataGenerator(rescale=1./255)

## LIFE LINE

In [36]:
life_train_generator = train_datagen.flow_from_dataframe(
    train_df,
    data_dir+'life',
    x_col='file',
    y_col='life',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

life_valid_generator = data_generator.flow_from_dataframe(
    valid_df,
    data_dir+'life',
    x_col='file',
    y_col='life',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

life_test_generator = data_generator.flow_from_dataframe(
    test_df,
    data_dir+'life',
    x_col='file',
    y_col='life',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

Found 758 validated image filenames belonging to 2 classes.
Found 95 validated image filenames belonging to 2 classes.
Found 95 validated image filenames belonging to 2 classes.


In [37]:
# CNN 모델 생성
life_model = models.Sequential()
life_model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size[0], img_size[1], 3)))
life_model.add(layers.MaxPooling2D((2, 2)))
life_model.add(layers.Conv2D(64, (3, 3), activation='relu'))
life_model.add(layers.MaxPooling2D((2, 2)))
life_model.add(layers.Conv2D(128, (3, 3), activation='relu'))
life_model.add(layers.MaxPooling2D((2, 2)))
life_model.add(layers.Conv2D(256, (3, 3), activation='relu'))
life_model.add(layers.MaxPooling2D((2, 2)))
life_model.add(layers.Flatten())
life_model.add(layers.Dense(512, activation='relu'))
life_model.add(layers.Dropout(0.5))
life_model.add(layers.Dense(256, activation='relu'))
life_model.add(layers.Dropout(0.5))
life_model.add(layers.Dense(1, activation='sigmoid'))

In [38]:
# 모델 컴파일
life_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [39]:
# 모델 학습
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

life_model_checkpoint_path = data_dir + 'seperated_augmented_life_model_checkpoint.h5'
early_stopping_patience = 5

early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=early_stopping_patience)
model_checkpoint = ModelCheckpoint(life_model_checkpoint_path, monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

life_history = life_model.fit(
    life_train_generator,
    steps_per_epoch=life_train_generator.samples // batch_size,
    epochs=30,
    validation_data=life_valid_generator,
    validation_steps=life_valid_generator.samples // batch_size,
    callbacks=[early_stopping, model_checkpoint]
)

Epoch 1/30
94/94 [==============================] - ETA: 0s - loss: 0.6705 - accuracy: 0.6120
Epoch 1: val_accuracy improved from -inf to 0.71591, saving model to /content/drive/MyDrive/PLSU/seperated_life_model_checkpoint.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


94/94 [==============================] - 25s 243ms/step - loss: 0.6705 - accuracy: 0.6120 - val_loss: 0.6573 - val_accuracy: 0.7159
Epoch 2/30
94/94 [==============================] - ETA: 0s - loss: 0.6702 - accuracy: 0.6240
Epoch 2: val_accuracy improved from 0.71591 to 0.72727, saving model to /content/drive/MyDrive/PLSU/seperated_life_model_checkpoint.h5
94/94 [==============================] - 27s 283ms/step - loss: 0.6702 - accuracy: 0.6240 - val_loss: 0.6110 - val_accuracy: 0.7273
Epoch 3/30
94/94 [==============================] - ETA: 0s - loss: 0.6580 - accuracy: 0.6307
Epoch 3: val_accuracy did not improve from 0.72727
94/94 [==============================] - 21s 223ms/step - loss: 0.6580 - accuracy: 0.6307 - val_loss: 0.5837 - val_accuracy: 0.7273
Epoch 4/30
94/94 [==============================] - ETA: 0s - loss: 0.6749 - accuracy: 0.6213
Epoch 4: val_accuracy did not improve from 0.72727
94/94 [==============================] - 22s 233ms/step - loss: 0.6749 - accuracy: 0.

In [40]:
from tensorflow.keras.models import load_model

life_loaded_model = load_model(life_model_checkpoint_path)
loss, accuracy = life_loaded_model.evaluate(life_test_generator, steps=len(life_test_generator))

12/12 [==============================] - 3s 223ms/step - loss: 0.8415 - accuracy: 0.6105


In [41]:
import numpy as np
from sklearn.metrics import f1_score

life_pred = life_loaded_model.predict(life_test_generator)
life_pred_classes = np.argmax(life_pred, axis=1)

life_true = life_test_generator.classes

life_f1 = f1_score(life_true, life_pred_classes, average='weighted')
print(life_f1)

12/12 [==============================] - 2s 191ms/step
0.4372255141164168


## HEAD LINE

In [24]:
head_train_generator = data_generator.flow_from_dataframe(
    train_df,
    data_dir+'head',
    x_col='file',
    y_col='head',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

head_valid_generator = data_generator.flow_from_dataframe(
    valid_df,
    data_dir+'head',
    x_col='file',
    y_col='head',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

head_test_generator = data_generator.flow_from_dataframe(
    test_df,
    data_dir+'head',
    x_col='file',
    y_col='head',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

Found 758 validated image filenames belonging to 2 classes.
Found 95 validated image filenames belonging to 2 classes.
Found 95 validated image filenames belonging to 2 classes.


In [25]:
# CNN 모델 생성
head_model = models.Sequential()
head_model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size[0], img_size[1], 3)))
head_model.add(layers.MaxPooling2D((2, 2)))
head_model.add(layers.Conv2D(64, (3, 3), activation='relu'))
head_model.add(layers.MaxPooling2D((2, 2)))
head_model.add(layers.Conv2D(128, (3, 3), activation='relu'))
head_model.add(layers.MaxPooling2D((2, 2)))
head_model.add(layers.Conv2D(256, (3, 3), activation='relu'))
head_model.add(layers.MaxPooling2D((2, 2)))
head_model.add(layers.Flatten())
head_model.add(layers.Dense(512, activation='relu'))
head_model.add(layers.Dropout(0.5))
head_model.add(layers.Dense(256, activation='relu'))
head_model.add(layers.Dropout(0.5))
head_model.add(layers.Dense(1, activation='sigmoid'))

In [26]:
# 모델 컴파일
head_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [27]:
# 모델 학습
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

head_model_checkpoint_path = data_dir + 'seperated_augmented_head_model_checkpoint.h5'
early_stopping_patience = 5

early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=early_stopping_patience)
model_checkpoint = ModelCheckpoint(head_model_checkpoint_path, monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

head_history = head_model.fit(
    head_train_generator,
    steps_per_epoch=head_train_generator.samples // batch_size,
    epochs=30,
    validation_data=head_valid_generator,
    validation_steps=head_valid_generator.samples // batch_size,
    callbacks=[early_stopping, model_checkpoint]
)

Epoch 1/30
94/94 [==============================] - ETA: 0s - loss: 0.6422 - accuracy: 0.6920
Epoch 1: val_accuracy improved from -inf to 0.70455, saving model to /content/drive/MyDrive/PLSU/seperated_head_model_checkpoint.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


94/94 [==============================] - 390s 4s/step - loss: 0.6422 - accuracy: 0.6920 - val_loss: 0.6063 - val_accuracy: 0.7045
Epoch 2/30
94/94 [==============================] - ETA: 0s - loss: 0.6249 - accuracy: 0.6973
Epoch 2: val_accuracy improved from 0.70455 to 0.71591, saving model to /content/drive/MyDrive/PLSU/seperated_head_model_checkpoint.h5
94/94 [==============================] - 25s 263ms/step - loss: 0.6249 - accuracy: 0.6973 - val_loss: 0.5963 - val_accuracy: 0.7159
Epoch 3/30
94/94 [==============================] - ETA: 0s - loss: 0.6186 - accuracy: 0.6920
Epoch 3: val_accuracy did not improve from 0.71591
94/94 [==============================] - 22s 227ms/step - loss: 0.6186 - accuracy: 0.6920 - val_loss: 0.6740 - val_accuracy: 0.7159
Epoch 4/30
94/94 [==============================] - ETA: 0s - loss: 0.6084 - accuracy: 0.6933
Epoch 4: val_accuracy did not improve from 0.71591
94/94 [==============================] - 24s 255ms/step - loss: 0.6084 - accuracy: 0.69

In [28]:
from tensorflow.keras.models import load_model

head_loaded_model = load_model(head_model_checkpoint_path)
loss, accuracy = head_loaded_model.evaluate(head_test_generator, steps=len(head_test_generator))

12/12 [==============================] - 32s 3s/step - loss: 0.5617 - accuracy: 0.7474


In [29]:
import numpy as np
from sklearn.metrics import f1_score

head_pred = head_loaded_model.predict(head_test_generator)
head_pred_classes = np.argmax(head_pred, axis=1)

head_true = head_test_generator.classes

head_f1 = f1_score(head_true, head_pred_classes, average='weighted')
print(head_f1)

12/12 [==============================] - 3s 224ms/step
0.6111681643132221


## HEART LINE

In [43]:
heart_train_generator = train_datagen.flow_from_dataframe(
    train_df,
    data_dir+'heart',
    x_col='file',
    y_col='heart',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

heart_valid_generator = data_generator.flow_from_dataframe(
    valid_df,
    data_dir+'heart',
    x_col='file',
    y_col='heart',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

heart_test_generator = data_generator.flow_from_dataframe(
    test_df,
    data_dir+'heart',
    x_col='file',
    y_col='heart',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

Found 758 validated image filenames belonging to 2 classes.
Found 95 validated image filenames belonging to 2 classes.
Found 95 validated image filenames belonging to 2 classes.


In [44]:
# CNN 모델 생성
heart_model = models.Sequential()
heart_model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size[0], img_size[1], 3)))
heart_model.add(layers.MaxPooling2D((2, 2)))
heart_model.add(layers.Conv2D(64, (3, 3), activation='relu'))
heart_model.add(layers.MaxPooling2D((2, 2)))
heart_model.add(layers.Conv2D(128, (3, 3), activation='relu'))
heart_model.add(layers.MaxPooling2D((2, 2)))
heart_model.add(layers.Conv2D(256, (3, 3), activation='relu'))
heart_model.add(layers.MaxPooling2D((2, 2)))
heart_model.add(layers.Flatten())
heart_model.add(layers.Dense(512, activation='relu'))
heart_model.add(layers.Dropout(0.5))
heart_model.add(layers.Dense(256, activation='relu'))
heart_model.add(layers.Dropout(0.5))
heart_model.add(layers.Dense(1, activation='sigmoid'))

In [45]:
# 모델 컴파일
heart_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [46]:
# 모델 학습
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

heart_model_checkpoint_path = data_dir + 'seperated_augmented_heart_model_checkpoint.h5'
early_stopping_patience = 5

early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=early_stopping_patience)
model_checkpoint = ModelCheckpoint(heart_model_checkpoint_path, monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

heart_history = heart_model.fit(
    heart_train_generator,
    steps_per_epoch=heart_train_generator.samples // batch_size,
    epochs=30,
    validation_data=heart_valid_generator,
    validation_steps=heart_valid_generator.samples // batch_size,
    callbacks=[early_stopping, model_checkpoint]
)

Epoch 1/30
94/94 [==============================] - ETA: 0s - loss: 0.6942 - accuracy: 0.5307
Epoch 1: val_accuracy improved from -inf to 0.50000, saving model to /content/drive/MyDrive/PLSU/seperated_augmented_heart_model_checkpoint.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


94/94 [==============================] - 24s 236ms/step - loss: 0.6942 - accuracy: 0.5307 - val_loss: 0.6974 - val_accuracy: 0.5000
Epoch 2/30
94/94 [==============================] - ETA: 0s - loss: 0.6911 - accuracy: 0.5400
Epoch 2: val_accuracy improved from 0.50000 to 0.52273, saving model to /content/drive/MyDrive/PLSU/seperated_augmented_heart_model_checkpoint.h5
94/94 [==============================] - 31s 333ms/step - loss: 0.6911 - accuracy: 0.5400 - val_loss: 0.6935 - val_accuracy: 0.5227
Epoch 3/30
94/94 [==============================] - ETA: 0s - loss: 0.6905 - accuracy: 0.5533
Epoch 3: val_accuracy did not improve from 0.52273
94/94 [==============================] - 22s 235ms/step - loss: 0.6905 - accuracy: 0.5533 - val_loss: 0.6983 - val_accuracy: 0.5114
Epoch 4/30
94/94 [==============================] - ETA: 0s - loss: 0.6881 - accuracy: 0.5533
Epoch 4: val_accuracy did not improve from 0.52273
94/94 [==============================] - 22s 230ms/step - loss: 0.6881 - a

In [47]:
from tensorflow.keras.models import load_model

heart_loaded_model = load_model(heart_model_checkpoint_path)
loss, accuracy = heart_loaded_model.evaluate(heart_test_generator, steps=len(heart_test_generator))

12/12 [==============================] - 3s 204ms/step - loss: 0.7648 - accuracy: 0.5789


In [48]:
import numpy as np
from sklearn.metrics import f1_score

heart_pred = heart_loaded_model.predict(heart_test_generator)
heart_pred_classes = np.argmax(heart_pred, axis=1)

heart_true = heart_test_generator.classes

heart_f1 = f1_score(heart_true, heart_pred_classes, average='weighted')
print(heart_f1)

12/12 [==============================] - 2s 179ms/step
0.26021671826625387
